# Modelos de datos en el dataset
 - Unique : ID único de 16 caracteres.
 - category : se deben proporcionar los valores categóricos
 - foreign : se debe especificar el dataset y la columna dataset.columna
 - date: fecha máxima:
 - numeric: tipo de dato numérico

**random**: Esto significa que el dataset tendrá como número máximo de registros la cantidad de valores en la columna con el mayor número de valores.

In [739]:
import pandas as pd
import numpy as np
import random
import uuid
from scipy.stats import truncnorm
from datetime import datetime

In [741]:
d1 = {
  "ds": "dataset1",
  "columns": [
    {
      "name": "departamento",
      "type": "category",
      "values": ["Venta", "Marketing", "Desarrollo", "Soporte"]
    },
    {
      "name": "id_empleados",
      "type": "unique"
    },
    {
      "name": "fecha_inicio",
      "type": "date",
      "values": {
          "min": "2010-12-31",
        "max": "2023-12-31"
      }
    },
    {
      "name": "sueldo",
      "type": "numeric",
      "values": {
        "min": 1200,
        "max": 70000
      }
    }
  ],
  "random": True,
  "random_rows": 1000
}

d2 = {
  "ds": "dataset2",
  "columns": [
    {
      "name": "region",
      "type": "category",
      "values": ["Norte", "Sur", "Este", "Oeste"]
    },
    {
      "name": "id_producto",
      "type": "unique"
    },
    {
      "name": "categoria_producto",
      "type": "category",
      "values": ["Electronica", "Mueble", "Ropa", "Comida"]
    },
    {
      "name": "Fecha_venta",
      "type": "date",
      "values": {
          "min": "2024-06-02",
        "max": "2024-06-30"
      }
    }
  ],
  "random": True,
  "random_rows": 1000
}



d3 = {
  "ds": "dataset3",
  "columns": [
    {
      "name": "Proyecto",
      "type": "category",
      "values": ["Alpha", "Beta", "Gamma", "Delta"]
    },
    {
      "name": "id_proyecto",
      "type": "unique"
    },
    {
      "name": "fecha_inicio",
      "type": "date",
      "values": {
        "min": "2024-12-01",
        "max": "2024-12-31"
      }
    },
    {
      "name": "presupuesto",
      "type": "numeric",
      "values": {
        "min": 100000,
        "max": 500000
      }
    },
    {
      "name": "id_administrador",
      "type": "foreign",
      "values": "dataset1.id_empleados"
    }
  ],
  "random": False,
}

d4={
  "ds": "dataset4",
  "columns": [
    {
      "name": "locacion",
      "type": "category",
      "values": ["Tegucigalpa", "Catacamas", "Danli", "Santa Rosa"]
    },
    {
      "name": "location_id",
      "type": "unique"
    },
    {
      "name": "fecha_establecimiento",
      "type": "date",
      "values": {
        "min":"2020-01-01",
        "max": "2020-02-01"
      }
    },
    {
      "name": "num_empleados",
      "type": "numeric",
      "values": {
        "min": 10,
        "max": 500
      }
    }
  ],
  "random": True,
  "random_rows": 1000
}
d5 = {
  "ds": "dataset5",
  "columns": [
      
    
    {
      "name": "producto_comprado",
      "type": "foreign",
      "values": "dataset2.categoria_producto"
    },
    {
      "name": "metodo_pago",
      "type": "category",
      "values": ["Tarjeta", "Efectivo", "Transferencia"]
    },
      {
      "name": "tipo_cliente",
      "type": "category",
      "values": ["Regular", "Nuevo", "Premium"]
    },
      {
          "name":"ubicacion",
          "type":"foreign",
          "values":"dataset2.region"
      }
  ,
    {
      "name": "total",
      "type": "numeric",
      "values": {
        "min": 100,
        "max": 500000
      }
    },
    
    {
      "name": "descuento",
      "type": "numeric",
      "values": {
        "min": 0,
        "max": 500
      }
    }],
  "random": True,
  "random_rows": 500
  
}



# Funciones 

In [743]:
#Esta función encuentra el tamaño máximo basado en la configuración proporcionada..

def findMax(config):
    size=0
    
    for column in config["columns"] :
        if column["type"] in ["date","numeric","category"]:
            if config["random"]:
                    size=config["random_rows"]
            else: 
                
                if column["type"]=="date":
                    
                    dateMin=datetime.strptime(column["values"]["min"],'%Y-%m-%d')
                    dateMax=datetime.strptime(column["values"]["max"],'%Y-%m-%d')
                    days=(dateMax-dateMin).days
                    if(days>size):
                        
                        size=days
                elif column["type"]=="category" :
                    
                    if len(column["values"])>size:
                        
                        size=len(column["values"])
                else:
                    if column["values"]["max"]!=None and column["values"]["max"]>size:
                        
                        size=column["values"]["max"]
                
                    
                
    return size
    
# Esta funcion genera n cantidad de ID de 16 caracteres
def generateID(n):
    list_id=[]
    for i in range(n):
        unique_id = uuid.uuid4().hex
        unique_id[16:]
        list_id.append(unique_id)
    return list_id
                
# Esta funcion genera n cantidad de fechas aleatorias en un rango determinado
def get_random_dates( min , max , n ):
    items = pd.date_range( start = min , end = max , freq = 'D' )
    return random.choices( items , k=n )                
                
            
# Esta funcion genera, una lista de tamanio n, dada una lista predeterminada     
def get_random_category(items,n):
    return random.choices( items , k=n )
        
# Esta funcion genera, n cantidad de valores numericos que siguen una funcion normal Truncada
  
def generate_truncated_normal_data(mean, std, min_val, max_val, size):
    # Calcular los parámetros de la distribución normal truncada
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)
    return data  
# Esta funcion genera, n cantidad de elementos que siguen la distribucion normal

def simpleNormal(min,max,n):
    list=[]
    mean=(min+max)/2
    std=(max-min)/6
    return np.random.normal(mean,std,n)


# Esta funcion, determina si existe un dataset y una determinada columna en el dataset
# Retorna la posicion del dataset en el config_list en caso de que exista
# Si no existe devuele un error

def searchDataset(config_list,name,column_name):
    for i in range(len(config_list)):
        if config_list[i]["ds"]==name:
            columns=[]
            for column in config_list[i]["columns"]:
                
                if column["name"]==column_name:
                    return i
            mensaje=f"No existe la columna \'{column_name}\' en el dataset  \'{name}\' las columnas que podria utilizar son : \n"+ str([ c["name"] for c in config_list[i]["columns"] ] )
            raise ValueError(mensaje)
    raise ValueError("No existe el \' dataset \' "+name)


In [745]:
#Esta funcion permite, construir un dataFrame  a partir de los archivos de configuracion
# recibe la lista de configuracion, la lista para los dataframe, la configuracion actual, y el tamaño del mismo
def build_dataframe(config_list,dataFrame_list,config):
    dataElement={}
    
    
    size=findMax(config)
    
    for column in config["columns"]: 
        if column["type"] in ["unique","category","foreign","date","numeric"]:

            if column["type"]=="foreign" :
                
                names=column["values"].split(".")
                
                index=searchDataset(config_list,names[0],names[1])

                if(dataFrame_list[index] is None):
                    dataFrame_list[index]=build_dataframe(config_list,dataFrame_list,config_list[index])
           
                dataElement[column["name"]]=random.choices(list(dataFrame_list[index][names[1]]),k=size )
                
            elif column["type"]=="unique" :
                    dataElement[column["name"]]=generateID(size)
                
            elif column["type"]== "date" :
                    dataElement[column["name"]]=get_random_dates(column["values"]["min"],column["values"]["max"],size)
                
            elif column["type"]=="category" :
                    dataElement[column["name"]]=get_random_category(column["values"],size)
                
            elif column["type"]=="numeric":
                if "mean" in column["values"].keys():
                    dataElement[column["name"]]=generate_truncated_normal_data(
                            column["values"]["mean"],column["values"]["std"],column["values"]["min"],column["values"]["max"],size)
                else:
                        dataElement[column["name"]]=simpleNormal(column["values"]["min"],column["values"]["max"],size)
                        
    return pd.DataFrame(dataElement)
    

In [698]:
def build_dataframes(config_list):
    dataFrameList=h=np.full(len(config_list),None)

    for i in range(len(config_list)):
        
        if dataFrameList[i] is None :
            dataFrameList[i]=build_dataframe(config_list,dataFrameList,config_list[i])
            
    return dataFrameList


    

In [749]:
config_list=[d1,d2,d3,d4,d5]

dataFrame_list=build_dataframes(config_list)



In [755]:
simulated_extended=dataFrame_list[4]

# Analisis de datos

In [757]:
simulated_extended

,producto_comprado,metodo_pago,tipo_cliente,ubicacion,total,descuento
0,Comida,Efectivo,Nuevo,Este,183237.723396,186.292719
1,Electronica,Tarjeta,Nuevo,Oeste,323806.639572,316.956534
2,Comida,Transferencia,Nuevo,Sur,276936.399670,273.213207
3,Comida,Transferencia,Nuevo,Este,153168.058125,369.398323
4,Electronica,Efectivo,Nuevo,Norte,258985.793735,294.200640
...,...,...,...,...,...,...
495,Comida,Tarjeta,Nuevo,Norte,111522.879374,353.269326
496,Electronica,Transferencia,Regular,Oeste,275259.193312,214.168107
497,Mueble,Efectivo,Regular,Sur,266049.786295,266.020183
498,Electronica,Tarjeta,Regular,Sur,137697.806678,234.820912


In [759]:
firstAgg = (
  simulated_extended

    .groupby( [
        "producto_comprado"
        , "metodo_pago"
        ,"tipo_cliente"
    ] , as_index=False )

    .agg({
        "ubicacion": ["count"]
    })
)

firstAgg.columns=["producto_comprado","metodo_pago","tipo_cliente","count"]


In [781]:
firstAgg['probs']=firstAgg['count']/simulated_extended.shape[0]
firstAgg.head(5)

,producto_comprado,metodo_pago,tipo_cliente,count,probs
0,Comida,Efectivo,Nuevo,6,0.012
1,Comida,Efectivo,Premium,10,0.020
2,Comida,Efectivo,Regular,7,0.014
3,Comida,Tarjeta,Nuevo,19,0.038
4,Comida,Tarjeta,Premium,19,0.038


In [779]:
k=10000


In [783]:
randIndex=np.random.choice(
firstAgg.index,
size=k,
p=list(firstAgg['probs'])
)



In [787]:
segment=firstAgg.loc[ randIndex ,["producto_comprado","metodo_pago","tipo_cliente" ]].loc[0]
segment

,producto_comprado,metodo_pago,tipo_cliente
0,Comida,Efectivo,Nuevo
0,Comida,Efectivo,Nuevo
0,Comida,Efectivo,Nuevo
0,Comida,Efectivo,Nuevo
0,Comida,Efectivo,Nuevo
...,...,...,...
0,Comida,Efectivo,Nuevo
0,Comida,Efectivo,Nuevo
0,Comida,Efectivo,Nuevo
0,Comida,Efectivo,Nuevo


In [789]:
simulated_extended.loc[
    (simulated_extended["producto_comprado"]=="Comida") 
&
(simulated_extended["metodo_pago"]=="Efectivo")
&
(simulated_extended["tipo_cliente"]=="Nuevo")
, :]



,producto_comprado,metodo_pago,tipo_cliente,ubicacion,total,descuento
0,Comida,Efectivo,Nuevo,Este,183237.723396,186.292719
120,Comida,Efectivo,Nuevo,Norte,89693.738395,379.649310
161,Comida,Efectivo,Nuevo,Oeste,298645.042199,111.186243
166,Comida,Efectivo,Nuevo,Norte,219449.068439,279.668899
336,Comida,Efectivo,Nuevo,Norte,276915.895192,329.027222
344,Comida,Efectivo,Nuevo,Norte,197009.388785,431.172140


In [793]:
simulated_extended.loc[
    (simulated_extended["producto_comprado"]=="Comida") 
&
(simulated_extended["metodo_pago"]=="Efectivo")
&
(simulated_extended["tipo_cliente"]=="Nuevo")
, :].shape[0]/simulated_extended.shape[0]

0.012

# aproximación para las estadisticas

In [803]:
segment.shape[0]/k

0.0127

# para las demas columnas 

In [805]:
def get_numeric_column_simulated( df_origin , categories , column_name,size ):
    
    randIndex=np.random.choice(
    firstAgg.index,
    size=size,
    p=list(firstAgg['probs'])
    )
    simulated=firstAgg.loc[randIndex,categories]

    a1 = df_origin.groupby(
        categories
        , as_index = False
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )
    nc = [ c for c in categories ]
    nc.extend( ["Min" , "Max" , "Mean" , "Std"] )
    a1.columns = nc

    ColumnSimulated = pd.DataFrame()
    for i in a1.index:
        rs = a1.loc[i]
        OneSegmnetCountryProduct = simulated.loc[i].copy()    
    
        data = generate_truncated_normal_data(
            rs["Mean"]
            , 1 if rs["Std"] == 0 else rs["Std"]
            , rs["Min"] - 1 if rs["Std"] == 0 else rs["Min"]
            , rs["Max"] + 1 if rs["Std"] == 0 else rs["Max"]
            , OneSegmnetCountryProduct.shape[0]
        )
    
        OneSegmnetCountryProduct[column_name] = data
    
        ColumnSimulated = pd.concat( [ ColumnSimulated , OneSegmnetCountryProduct  ] )

    return ColumnSimulated.reset_index(drop=True)
    

In [811]:
k=10000

In [813]:
categories = [ "producto_comprado" , "metodo_pago" , "tipo_cliente" ]
numeric_columns = ["total","descuento"]

final_simulation = simulated.sort_index().reset_index(drop=True).copy()

for nc in numeric_columns:
    ncdf = get_numeric_column_simulated( simulated_extended , categories , nc,k )

    final_simulation = pd.merge( final_simulation , ncdf.loc[ : , [nc] ] , left_index=True , right_index=True )

final_simulation

,producto_comprado,metodo_pago,tipo_cliente,total,descuento
0,Comida,Efectivo,Nuevo,234927.105555,256.368005
1,Comida,Efectivo,Nuevo,208521.360770,257.317332
2,Comida,Efectivo,Nuevo,212224.630766,301.001271
3,Comida,Efectivo,Nuevo,287579.311712,281.939887
4,Comida,Efectivo,Nuevo,139524.501186,399.950783
...,...,...,...,...,...
9995,Ropa,Transferencia,Regular,220617.583813,149.628007
9996,Ropa,Transferencia,Regular,209982.553669,259.160767
9997,Ropa,Transferencia,Regular,288725.095766,198.084792
9998,Ropa,Transferencia,Regular,286689.044322,321.472821
